In [33]:
import os
import json
import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import gensim

import sys
from nltk.tokenize import sent_tokenize

from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import ast
from nltk.corpus import wordnet

In [34]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jengyuchou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jengyuchou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jengyuchou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jengyuchou/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [35]:
sqlite_path = "./reddit_ed_0.6.0.sqlite3"
engine = create_engine(
    f"sqlite:///{sqlite_path}",
    connect_args={"check_same_thread":False},
)

In [36]:
all_subreddits_polarity_df = pd.read_csv('submission_polarity.csv')

In [37]:
all_subreddits_polarity_df

,id,selftext,sentence_polarity,subreddit
0,10pljws,Is it possible to gain 5 pounds in 10 days? I’...,0.938680,AnorexiaRecovery
1,10pg79q,"Hello all, not sure if this is the best place ...",0.149335,AnorexiaRecovery
2,10p9kga,I’m soo proud of myself 🥲,6.462965,AnorexiaRecovery
3,10p8r2g,do i need to be concerned about chest pain whe...,0.504343,AnorexiaRecovery
4,10p6vc9,"2 weeks ago is was 58 lbs, I got on the scale ...",0.409928,AnorexiaRecovery
...,...,...,...,...
16664,lrjuuq,"So I recently lost a pet, today is the 1 year ...",0.232494,NotOtherwiseSpecified
16665,lpje44,Who the fuck decided that “FED” should be in t...,-0.040827,NotOtherwiseSpecified
16666,lorrkh,I suck at making them but I love a good meme! ...,-0.012205,NotOtherwiseSpecified
16667,lorq56,I’m so tired of feeling like a “fake anorexic”...,-0.004249,NotOtherwiseSpecified


In [38]:
query = f"select id, created_utc from subreddit_submission_metadata;"
df = pd.read_sql_query(text(query),con=engine.connect())

In [39]:
df = df.merge(all_subreddits_polarity_df,how='inner',on='id')

In [40]:
df

,id,created_utc,selftext,sentence_polarity,subreddit
0,10poqxu,1675140001,23 F. I just started treatment for an eating d...,-1.895242,EatingDisorders
1,10pnoxo,1675137504,I 26 f told the truth for the first time about...,-1.581629,EatingDisorders
2,10pkd8e,1675128549,"Greetings,\n\nA research team from Carnegie Va...",-0.117162,EatingDisorders
3,10pjinj,1675126245,Idk who to talk to about this so I resort to r...,-2.159229,EatingDisorders
4,10pj90n,1675125516,I don’t know where to start.. Due to my upbrin...,-1.407739,EatingDisorders
...,...,...,...,...,...
16664,lrjuuq,1614191529,"So I recently lost a pet, today is the 1 year ...",0.232494,NotOtherwiseSpecified
16665,lpje44,1613983999,Who the fuck decided that “FED” should be in t...,-0.040827,NotOtherwiseSpecified
16666,lorrkh,1613892434,I suck at making them but I love a good meme! ...,-0.012205,NotOtherwiseSpecified
16667,lorq56,1613892304,I’m so tired of feeling like a “fake anorexic”...,-0.004249,NotOtherwiseSpecified


In [41]:
from datetime import datetime

In [42]:
datetime.fromtimestamp(675140001)

datetime.datetime(1991, 5, 24, 22, 53, 21)

In [80]:
before = datetime(2023,1,31,0,0)  # Jan 1, 2021

In [81]:
after = datetime(2015,1,1,0,0) # Jan 31, 2019

In [82]:
# if submission was posted in the date range of after-before, 2020, ignore

In [83]:
df[['id','created_utc']]

,id,created_utc
0,10poqxu,1675140001
1,10pnoxo,1675137504
2,10pkd8e,1675128549
3,10pjinj,1675126245
4,10pj90n,1675125516
...,...,...
16664,lrjuuq,1614191529
16665,lpje44,1613983999
16666,lorrkh,1613892434
16667,lorq56,1613892304


In [84]:
df_ts = list(zip(df['id'].tolist(),df['created_utc'].tolist()))

In [98]:
non_2020 = list()

In [99]:
for i in df_ts:
    current = datetime.fromtimestamp(i[1])
    if current.year != 2020:
        non_2020.append(i[0])


In [100]:
len(df_ts)

16669

In [101]:
len(non_2020)

13364

In [111]:
df[df['id'] is in]

,id,created_utc,selftext,sentence_polarity,subreddit
0,10poqxu,1675140001,23 F. I just started treatment for an eating d...,-1.895242,EatingDisorders
1,10pnoxo,1675137504,I 26 f told the truth for the first time about...,-1.581629,EatingDisorders
2,10pkd8e,1675128549,"Greetings,\n\nA research team from Carnegie Va...",-0.117162,EatingDisorders
3,10pjinj,1675126245,Idk who to talk to about this so I resort to r...,-2.159229,EatingDisorders
4,10pj90n,1675125516,I don’t know where to start.. Due to my upbrin...,-1.407739,EatingDisorders
...,...,...,...,...,...
16664,lrjuuq,1614191529,"So I recently lost a pet, today is the 1 year ...",0.232494,NotOtherwiseSpecified
16665,lpje44,1613983999,Who the fuck decided that “FED” should be in t...,-0.040827,NotOtherwiseSpecified
16666,lorrkh,1613892434,I suck at making them but I love a good meme! ...,-0.012205,NotOtherwiseSpecified
16667,lorq56,1613892304,I’m so tired of feeling like a “fake anorexic”...,-0.004249,NotOtherwiseSpecified
